# Storage

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows=10
pd.__version__

In [ ]:
df = pd.DataFrame({'A' : range(4), 
                   'B' : 1.0, 
                   'C' : 'foo', 
                   'D' : pd.Timestamp('20130101'), 
                   'E' : 2.0})

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df._data

## Why do we have this arrangement?

In [ ]:
arr = np.random.randn(1000000, 20)

In [ ]:
df = DataFrame(arr)
df.info()

Constructing direct from a np.array

In [ ]:
%timeit DataFrame(arr)

Ufunc operation

In [ ]:
result_blocked = df.sum()
result_blocked

In [ ]:
%timeit df.sum()

In [ ]:
%timeit df.sum(1)

Construct a columnar layout

In [ ]:
d = [ Series(v.copy()) for c, v in df.iteritems() ]

In [ ]:
def f(d):
    return Series([ e.sum() for e in d])
result_columnar = f(d)
result_columnar

In [ ]:
result_columnar.equals(result_blocked)

## We are *cache* friendly!

In [ ]:
%timeit f(d)

Can't we just use *views*?

In [ ]:
d = [ Series(v) for c, v in df.iteritems() ]

But this is NOT friendly

In [ ]:
%timeit f(d)

Construct a column like DataFrame

In [ ]:
%timeit [ Series(v.copy()) for c, v in df.iteritems() ]

Construct a frame from columnar, with blocking

In [ ]:
%timeit DataFrame({c:Series(v) for c, v in df.iteritems()})

## Further Reading

http://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays/